# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-11-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-11-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-12 05:26:19,33.93911,67.709953,48116,1945,38141,8030.0,Afghanistan,123.601466,4.042314
1,NaN,NaN,NaN,Albania,2020-12-12 05:26:19,41.15330,20.168300,46863,977,24136,21750.0,Albania,1628.431441,2.084800
2,NaN,NaN,NaN,Algeria,2020-12-12 05:26:19,28.03390,1.659600,91121,2575,59590,28956.0,Algeria,207.796654,2.825913
3,NaN,NaN,NaN,Andorra,2020-12-12 05:26:19,42.50630,1.521800,7236,78,6598,560.0,Andorra,9365.171811,1.077944
4,NaN,NaN,NaN,Angola,2020-12-12 05:26:19,-11.20270,17.873900,16061,365,8798,6898.0,Angola,48.867733,2.272586


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,46718,46837,46837,47072,47306,47516,47716,47851,48053,48116
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,39719,40501,41302,42148,42988,43683,44436,45188,46061,46863
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,85084,85927,86730,87502,88252,88825,89416,90014,90579,91121


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1841,1846,1846,1864,1874,1900,1906,1919,1935,1945
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,839,852,870,889,905,922,936,951,965,977
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2464,2480,2492,2501,2516,2527,2539,2554,2564,2575


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37218,37260,37260,37393,37685,37879,37920,38032,38099,38141
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,19912,20484,20974,21286,21617,22180,22527,23072,23609,24136
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,54990,55538,56079,56617,57146,57648,58146,58146,59135,59590


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2873,2893,2945,2979,3005,3043,3087,3117,3186,3233
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,9163,9341,9501,9626,9728,9821,9974,10087,10288,10489
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1206,1214,1217,1219,1223,1224,1240,1245,1258,1264


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,42,42,42,42,42,42,42,42,42,42
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,137,137,138,138,138,138,141,141,141,141
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,29,29,29,29,29,29,30,30,30,30


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
649,1001.0,Autauga,Alabama,US,2020-12-12 05:26:19,32.539527,-86.644082,3233,42,0,3191.0,"Autauga, Alabama, US",5786.751150,1.299103
686,1075.0,Lamar,Alabama,US,2020-12-12 05:26:19,33.779950,-88.096680,830,9,0,821.0,"Lamar, Alabama, US",6012.314379,1.084337
687,1077.0,Lauderdale,Alabama,US,2020-12-12 05:26:19,34.901719,-87.656247,4970,60,0,4910.0,"Lauderdale, Alabama, US",5359.704084,1.207243


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,15842789,295450,6135314
India,9826775,142628,9324328
Brazil,6836227,180437,6078287


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,15842789,295450,6135314,9412025,2020-12-12 05:26:19,37.936303,-91.379001
India,9826775,142628,9324328,359819,2020-12-12 05:26:19,23.088275,81.806127
Brazil,6836227,180437,6078287,577503,2020-12-12 05:26:19,-12.669522,-48.480493
Russia,2574319,45370,2041006,487943,2020-12-12 05:26:19,54.546312,62.120860
France,2405210,57671,181581,2165958,2020-12-12 05:26:19,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,1516215,20787,0
Texas,1388909,24143,0
Florida,1106396,19714,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,1516215,20787,0,1495428,2020-12-12 05:26:19,37.843962,-120.728594
Texas,1388909,24143,0,1364766,2020-12-12 05:26:19,31.660643,-98.653069
Florida,1106396,19714,0,1086682,2020-12-12 05:26:19,28.940755,-82.700744
Illinois,832951,15067,0,817884,2020-12-12 05:26:19,39.843618,-89.178330
New York,753837,35357,0,718480,2020-12-12 05:26:19,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,501635,8199,0
Illinois,Cook,346004,7282,0
Florida,Miami-Dade,253403,3959,0
Arizona,Maricopa,245671,4299,0
Texas,Harris,204850,3128,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,501635,8199,0,493436,2020-12-12 05:26:19,34.308284,-118.228241,6037.0
Illinois,Cook,346004,7282,0,338722,2020-12-12 05:26:19,41.841448,-87.816588,17031.0
Florida,Miami-Dade,253403,3959,0,249444,2020-12-12 05:26:19,25.611236,-80.551706,12086.0
Arizona,Maricopa,245671,4299,0,241372,2020-12-12 05:26:19,33.348359,-112.491815,4013.0
Texas,Harris,204850,3128,0,201722,2020-12-12 05:26:19,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,47851,45188,90014,7162,15804,146,1475222,144066,28000,311002,...,1771545,8104,74352,1,105852,1381,102992,2079,18062,11007
2020-12-10,48053,46061,90579,7190,15925,146,1482216,145240,28011,313688,...,1792611,8487,74498,1,106280,1385,104879,2081,18091,11081
2020-12-11,48116,46863,91121,7236,16061,147,1489328,146317,28025,316581,...,1814395,8849,74811,1,106715,1391,106622,2082,18161,11162


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,1919,951,2554,78,358,4,40222,2393,908,4056,...,62663,87,611,0,933,35,890,606,364,304
2020-12-10,1935,965,2564,78,362,4,40431,2416,908,4163,...,63179,90,611,0,938,35,910,606,364,305
2020-12-11,1945,977,2575,78,365,4,40606,2445,908,4289,...,63603,90,612,0,944,35,931,606,365,306


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,38032,23072,58146,6452,8579,138,1311488,120532,25664,265815,...,3778,5564,71740,1,100952,1225,76893,1383,17295,9147
2020-12-10,38099,23609,59135,6505,8679,138,1318187,121882,25665,269665,...,3787,5747,71740,1,101384,1225,78715,1383,17307,9253
2020-12-11,38141,24136,59590,6598,8798,138,1324792,123474,25671,273503,...,3799,5941,72084,1,101823,1238,81166,1383,17329,9324


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2873,2893,2945,2979,3005,3043,3087,3117,3186,3233
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,713,725,734,744,755,759,769,786,806,830
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,4360,4433,4522,4593,4626,4662,4722,4805,4917,4970
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1353,1373,1399,1421,1422,1431,1453,1480,1525,1541
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,7925,8015,8071,8131,8171,8218,8298,8386,8499,8595


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-09,280187,38707,0,382601,176652,1448987,272346,140548,42393,49,...,422962,1359951,222545,5285,1698,267128,189863,58462,452180,37885
2020-12-10,284922,39345,0,387529,178854,1482551,276995,142979,43036,49,...,428973,1374256,225946,5413,1733,271043,192413,59695,456889,38223
2020-12-11,288775,39975,0,394512,181624,1516215,281673,146761,43818,49,...,436262,1388909,228129,5541,1791,274438,195554,60637,461367,38785


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-09        3117   10087    1245  1322   3496     722   1263    7392   
2020-12-10        3186   10288    1258  1359   3600     722   1287    7534   
2020-12-11        3233   10489    1264  1398   3663     723   1289    7658   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-09         1961     1134  ...    1552    375     2237      526   
2020-12-10         1977     1149  ...    1574    373     2267      534   
2020-12-11         1982     1167  ...    1595    378     2290      536   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-09           2245  1834  1250          0      586    427  
2020-12-10           2281  1852  1271          0      596    428  
2020-12-11           2376  1868  1281          0      620    428  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-09,3985,150,0,7081,2786,20411,3639,5285,803,0,...,5171,23657,995,86,23,4281,3016,901,4142,299
2020-12-10,4034,155,0,7154,2820,20603,3759,5327,806,0,...,5240,23897,1016,89,23,4335,2850,921,4209,299
2020-12-11,4086,158,0,7245,2875,20787,3846,5363,807,0,...,5327,24143,1025,93,23,4370,2879,938,4266,321


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-09          42     141      30   39     47      20     42     127   
2020-12-10          42     141      30   40     47      20     42     129   
2020-12-11          42     141      30   40     47      20     44     129   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-09           55       24  ...       7      8       13        4   
2020-12-10           55       24  ...       7      8       13        4   
2020-12-11           55       24  ...       7      8       13        5   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-09             13     2     7          0       11      2  
2020-12-10             13     2     7          0       11      2  
2020-12-11             14     2     7          0       11      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,0.002829,0.016923,0.006688,0.004911,0.004768,0.000000,0.003608,0.007962,0.000250,0.009517,...,0.009479,0.038176,0.001967,0.0,0.004441,0.002905,0.018623,0.000481,0.005511,0.008706
2020-12-10,0.004221,0.019319,0.006277,0.003910,0.007656,0.000000,0.004741,0.008149,0.000393,0.008637,...,0.011891,0.047261,0.001964,0.0,0.004043,0.002896,0.018322,0.000962,0.001606,0.006723
2020-12-11,0.001311,0.017412,0.005984,0.006398,0.008540,0.006849,0.004798,0.007415,0.000500,0.009223,...,0.012152,0.042653,0.004201,0.0,0.004093,0.004332,0.016619,0.000481,0.003869,0.007310


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,0.006821,0.016026,0.005908,0.0,0.008451,0.0,0.005324,0.008853,0.0,0.013493,...,0.008579,0.011628,0.000000,NaN,0.005388,0.0,0.026528,0.0,0.000000,0.003300
2020-12-10,0.008338,0.014721,0.003915,0.0,0.011173,0.0,0.005196,0.009611,0.0,0.026381,...,0.008235,0.034483,0.000000,NaN,0.005359,0.0,0.022472,0.0,0.000000,0.003289
2020-12-11,0.005168,0.012435,0.004290,0.0,0.008287,0.0,0.004328,0.012003,0.0,0.030267,...,0.006711,0.000000,0.001637,NaN,0.006397,0.0,0.023077,0.0,0.002747,0.003279


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,0.002954,0.024193,0.000000,0.013350,0.012869,0.029851,0.004520,0.011624,0.000507,0.013347,...,0.003453,0.027516,0.003609,0.0,0.004737,0.000817,0.021739,0.000724,0.001564,0.009380
2020-12-10,0.001762,0.023275,0.017009,0.008215,0.011656,0.000000,0.005108,0.011200,0.000039,0.014484,...,0.002382,0.032890,0.000000,0.0,0.004279,0.000000,0.023695,0.000000,0.000694,0.011588
2020-12-11,0.001102,0.022322,0.007694,0.014297,0.013711,0.000000,0.005011,0.013062,0.000234,0.014232,...,0.003169,0.033757,0.004795,0.0,0.004330,0.010612,0.031138,0.000000,0.001271,0.007673


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-09,0.012730,0.015505,NaN,0.011752,0.013349,0.023733,0.013988,0.016563,0.022405,0.0,...,0.019802,0.009882,0.011702,0.020270,0.010714,0.016740,0.013538,0.024571,0.009310,0.010941
2020-12-10,0.016899,0.016483,NaN,0.012880,0.012465,0.023164,0.017070,0.017297,0.015168,0.0,...,0.014212,0.010519,0.015282,0.024219,0.020612,0.014656,0.013431,0.021091,0.010414,0.008922
2020-12-11,0.013523,0.016012,NaN,0.018019,0.015487,0.022707,0.016888,0.026451,0.018171,0.0,...,0.016992,0.010662,0.009662,0.023647,0.033468,0.012526,0.016324,0.015780,0.009801,0.014703


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-09      0.009718  0.011329  0.004032  0.003797  0.020432  0.004172   
2020-12-10      0.022137  0.019927  0.010442  0.027988  0.029748  0.000000   
2020-12-11      0.014752  0.019537  0.004769  0.028698  0.017500  0.001385   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-09      0.004773  0.012603  0.008226  0.008000  ...  0.020381   
2020-12-10      0.019002  0.019210  0.008159  0.013228  ...  0.014175   
2020-12-11      0.001554  0.016459  0.002529  0.015666  ...  0.013342   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-09      0.016260  0.010845  0.019380   0.020455  0.006586  0.010509   
2020-12-10     -0.005333  0.013411  0.015209   0.016036  0.009815  0.016800   
2020-12-11      0.013405  0.010146  0.003745   0.041648  0.008639  0.007868   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-09            NaN  0.015598  0.004706  
2020-12-10            NaN  0.017065  0.002342  
2020-12-11            NaN  0.040268  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-09,0.010908,0.027397,NaN,0.015488,0.012355,0.010195,0.079181,0.008203,0.000000,NaN,...,0.012135,0.012194,0.023663,0.011765,0.0,0.004930,0.016515,0.035632,0.021707,0.034602
2020-12-10,0.012296,0.033333,NaN,0.010309,0.012204,0.009407,0.032976,0.007947,0.003736,NaN,...,0.013344,0.010145,0.021106,0.034884,0.0,0.012614,-0.055040,0.022198,0.016176,0.000000
2020-12-11,0.012890,0.019355,NaN,0.012720,0.019504,0.008931,0.023144,0.006758,0.001241,NaN,...,0.016603,0.010294,0.008858,0.044944,0.0,0.008074,0.010175,0.018458,0.013542,0.073579


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                        \
Admin2         Autauga Baldwin Barbour      Bibb    Blount Bullock    Butler   
2020-12-09         0.0     0.0     0.0  0.000000  0.021739     0.0  0.000000   
2020-12-10         0.0     0.0     0.0  0.025641  0.000000     0.0  0.000000   
2020-12-11         0.0     0.0     0.0  0.000000  0.000000     0.0  0.047619   

Province_State                               ... Wyoming                   \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte  Sheridan   
2020-12-09      0.000000  0.018519      0.0  ...     0.0    0.0  0.083333   
2020-12-10      0.015748  0.000000      0.0  ...     0.0    0.0  0.000000   
2020-12-11      0.000000  0.000000      0.0  ...     0.0    0.0  0.000000   

Province_State                                                                  
Admin2          Sublette Sweetwater Teton     Uinta Unassigned Washakie Weston  
2020-12-09      0.333333   0.181818   0.0  0.166667       -1.0      0.1    0.0  
2020-12-10      0.000000   0.000000   0.0  0.000000        NaN      0.0    0.0  
2020-12-11      0.250000   0.076923   0.0  0.000000        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,0.003544,0.017287,0.006946,0.005446,0.004636,0.001806,0.003100,0.006349,0.000282,0.008978,...,0.008801,0.037893,0.001848,0.0,0.004280,0.003559,0.017366,-0.027565,0.003579,0.008715
2020-12-10,0.003883,0.018303,0.006611,0.004678,0.006146,0.000903,0.003921,0.007249,0.000337,0.008807,...,0.010346,0.042577,0.001906,0.0,0.004162,0.003228,0.017844,-0.013301,0.002592,0.007719
2020-12-11,0.002597,0.017858,0.006298,0.005538,0.007343,0.003876,0.004359,0.007332,0.000419,0.009015,...,0.011249,0.042615,0.003054,0.0,0.004127,0.003780,0.017232,-0.006410,0.003231,0.007514


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,0.006686,0.016557,0.005358,0.000509,0.005119,4.967054e-09,0.004259,0.009640,5.383476e-07,0.018622,...,0.007806,0.017686,0.000007,NaN,0.004900,1.254929e-31,0.024382,-0.013585,0.000306,0.011856
2020-12-10,0.007512,0.015639,0.004637,0.000254,0.008146,2.483527e-09,0.004728,0.009626,2.691738e-07,0.022501,...,0.008020,0.026085,0.000004,NaN,0.005129,6.274644e-32,0.023427,-0.006793,0.000153,0.007573
2020-12-11,0.006340,0.014037,0.004463,0.000127,0.008217,1.241763e-09,0.004528,0.010815,1.345869e-07,0.026384,...,0.007366,0.013042,0.000820,NaN,0.005763,3.137322e-32,0.023252,-0.003396,0.001450,0.005426


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-09,0.003041,0.021499,0.004458,0.011949,0.010662,0.015956,0.004398,0.011045,0.000345,0.016321,...,0.005032,0.029535,0.002760,0.0,0.004734,0.001062,0.019657,-0.026408,0.002011,0.009122
2020-12-10,0.002401,0.022387,0.010734,0.010082,0.011159,0.007978,0.004753,0.011122,0.000192,0.015402,...,0.003707,0.031213,0.001380,0.0,0.004506,0.000531,0.021676,-0.013204,0.001352,0.010355
2020-12-11,0.001752,0.022355,0.009214,0.012189,0.012435,0.003989,0.004882,0.012092,0.000213,0.014817,...,0.003438,0.032485,0.003088,0.0,0.004418,0.005572,0.026407,-0.006602,0.001312,0.009014


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-09,0.012906,0.016118,NaN,0.016869,0.012296,0.021022,0.014751,0.021335,0.020859,7.177667e-82,...,0.017356,0.009773,0.011632,0.020075,0.012016,0.015738,0.016882,0.022201,0.009174,0.012487
2020-12-10,0.014903,0.016300,NaN,0.014874,0.012381,0.022093,0.015911,0.019316,0.018013,3.588833e-82,...,0.015784,0.010146,0.013457,0.022147,0.016314,0.015197,0.015156,0.021646,0.009794,0.010704
2020-12-11,0.014213,0.016156,NaN,0.016447,0.013934,0.022400,0.016399,0.022883,0.018092,1.794417e-82,...,0.016388,0.010404,0.011559,0.022897,0.024891,0.013861,0.015740,0.018713,0.009798,0.012704


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-09      0.011415  0.012349  0.005811  0.007196  0.020286  0.004193   
2020-12-10      0.016776  0.016138  0.008127  0.017592  0.025017  0.002096   
2020-12-11      0.015764  0.017838  0.006448  0.023145  0.021258  0.001741   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-09      0.006983  0.013796  0.009278  0.010092  ...  0.018353   
2020-12-10      0.012993  0.016503  0.008719  0.011660  ...  0.016264   
2020-12-11      0.007273  0.016481  0.005624  0.013663  ...  0.014803   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-09      0.016054  0.010320  0.018145   0.018423  0.009095  0.012558   
2020-12-10      0.005361  0.011865  0.016677   0.017229  0.009455  0.014679   
2020-12-11      0.009383  0.011006  0.010211   0.029439  0.009047  0.011273   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-09           -1.0  0.023931  0.005073  
2020-12-10           -1.0  0.020498  0.003707  
2020-12-11           -1.0  0.030383  0.001854  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-09,0.009809,0.018519,NaN,0.009269,0.013961,0.008655,0.041432,0.007055,0.004122,NaN,...,0.013935,0.009227,0.020052,0.022955,1.007926e-12,0.006266,0.011534,0.029229,0.018258,0.029213
2020-12-10,0.011052,0.025926,NaN,0.009789,0.013082,0.009031,0.037204,0.007501,0.003929,NaN,...,0.013639,0.009686,0.020579,0.028919,5.039628e-13,0.009440,-0.021753,0.025713,0.017217,0.014607
2020-12-11,0.011971,0.022641,NaN,0.011255,0.016293,0.008981,0.030174,0.007130,0.002585,NaN,...,0.015121,0.009990,0.014719,0.036932,2.519814e-13,0.008757,-0.005789,0.022086,0.015380,0.044093


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                          \
Admin2               Autauga   Baldwin   Barbour      Bibb    Blount   
2020-12-09      3.083806e-06  0.006331  0.015062  0.007745  0.011877   
2020-12-10      1.541903e-06  0.003166  0.007531  0.016693  0.005939   
2020-12-11      7.709516e-07  0.001583  0.003765  0.008346  0.002969   

Province_State                                                        ...  \
Admin2           Bullock    Butler   Calhoun  Chambers      Cherokee  ...   
2020-12-09      0.000822  0.000012  0.000905  0.010337  2.543132e-07  ...   
2020-12-10      0.000411  0.000006  0.008326  0.005168  1.271566e-07  ...   
2020-12-11      0.000206  0.023813  0.004163  0.002584  6.357830e-08  ...   

Province_State   Wyoming                                           \
Admin2              Park    Platte  Sheridan  Sublette Sweetwater   
2020-12-09      0.020837  0.000286  0.053812  0.236979   0.108618   
2020-12-10      0.010419  0.000143  0.026906  0.118490   0.054309   
2020-12-11      0.005209  0.000071  0.013453  0.184245   0.065616   

Province_State                                                         
Admin2                 Teton     Uinta Unassigned  Washakie    Weston  
2020-12-09      7.275958e-12  0.110286  -0.501961  0.081254  0.001953  
2020-12-10      3.637979e-12  0.055143  -0.501961  0.040627  0.000977  
2020-12-11      1.818989e-12  0.027572  -0.501961  0.020314  0.000488  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201211,AK,38584,NaN,1084961,NaN,totalTestsViral,1123545,146.0,869.0,...,1123545,3,9,47f2cb8f9f01019549b85b0450c8c42c15bb3e77,0,0,0,0,0,NaN
1,20201211,AL,288775,50896.0,1458560,NaN,totalTestsPeopleViral,1696439,2111.0,28146.0,...,1747335,52,528,3bf1ccc167c9d139e7506942ea8f53c42bdeeab7,0,0,0,0,0,NaN
2,20201211,AR,181624,25613.0,1672168,NaN,totalTestsViral,1828179,1059.0,9848.0,...,1853792,55,139,b15164e19ff0d50f1226dd7e1d8a1044dbd302c8,0,0,0,0,0,NaN
3,20201211,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,764d10a732e70e3fd761b0b673a6ac9775f3f823,0,0,0,0,0,NaN
4,20201211,AZ,394512,14561.0,2080870,NaN,totalTestsViral,4187860,3482.0,30302.0,...,2475382,91,967,8e92e841762001166ff76fda907512fdc4b5970c,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201211,AK,38584.0,NaN,1084961.0,NaN,totalTestsViral,1123545.0,146.0,869.0,...,1123545,3,9,47f2cb8f9f01019549b85b0450c8c42c15bb3e77,0,0,0,0,0,NaN
1,20201211,AL,288775.0,50896.0,1458560.0,NaN,totalTestsPeopleViral,1696439.0,2111.0,28146.0,...,1747335,52,528,3bf1ccc167c9d139e7506942ea8f53c42bdeeab7,0,0,0,0,0,NaN
2,20201211,AR,181624.0,25613.0,1672168.0,NaN,totalTestsViral,1828179.0,1059.0,9848.0,...,1853792,55,139,b15164e19ff0d50f1226dd7e1d8a1044dbd302c8,0,0,0,0,0,NaN
3,20201211,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,764d10a732e70e3fd761b0b673a6ac9775f3f823,0,0,0,0,0,NaN
4,20201211,AZ,394512.0,14561.0,2080870.0,NaN,totalTestsViral,4187860.0,3482.0,30302.0,...,2475382,91,967,8e92e841762001166ff76fda907512fdc4b5970c,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-11,AK,38584,NaN,1084961,NaN,totalTestsViral,1123545,146.0,869.0,NaN,...,1123545,3,9,47f2cb8f9f01019549b85b0450c8c42c15bb3e77,0,0,0,0,0,NaN
2020-12-11,AL,288775,50896.0,1458560,NaN,totalTestsPeopleViral,1696439,2111.0,28146.0,NaN,...,1747335,52,528,3bf1ccc167c9d139e7506942ea8f53c42bdeeab7,0,0,0,0,0,NaN
2020-12-11,AR,181624,25613.0,1672168,NaN,totalTestsViral,1828179,1059.0,9848.0,372.0,...,1853792,55,139,b15164e19ff0d50f1226dd7e1d8a1044dbd302c8,0,0,0,0,0,NaN
2020-12-11,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,764d10a732e70e3fd761b0b673a6ac9775f3f823,0,0,0,0,0,NaN
2020-12-11,AZ,394512,14561.0,2080870,NaN,totalTestsViral,4187860,3482.0,30302.0,809.0,...,2475382,91,967,8e92e841762001166ff76fda907512fdc4b5970c,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-11,AK,38584.0,NaN,1084961.0,NaN,totalTestsViral,1123545.0,146.0,869.0,NaN,...,1123545,3,9,47f2cb8f9f01019549b85b0450c8c42c15bb3e77,0,0,0,0,0,NaN
2020-12-11,AL,288775.0,50896.0,1458560.0,NaN,totalTestsPeopleViral,1696439.0,2111.0,28146.0,NaN,...,1747335,52,528,3bf1ccc167c9d139e7506942ea8f53c42bdeeab7,0,0,0,0,0,NaN
2020-12-11,AR,181624.0,25613.0,1672168.0,NaN,totalTestsViral,1828179.0,1059.0,9848.0,372.0,...,1853792,55,139,b15164e19ff0d50f1226dd7e1d8a1044dbd302c8,0,0,0,0,0,NaN
2020-12-11,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,764d10a732e70e3fd761b0b673a6ac9775f3f823,0,0,0,0,0,NaN
2020-12-11,AZ,394512.0,14561.0,2080870.0,NaN,totalTestsViral,4187860.0,3482.0,30302.0,809.0,...,2475382,91,967,8e92e841762001166ff76fda907512fdc4b5970c,0,0,0,0,0,NaN
